In [14]:
!pip install spotipy

In [15]:
import time
import numpy as np
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from config import *
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [16]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))

In [17]:
url = "https://www.billboard.com/charts/hot-100"
download = requests.get(url)
top_100 = BeautifulSoup(download.text, "html.parser")

In [18]:
# After inspecting the website on Chrome, found the information we're looking at under the tag: span > chart-element__information
top_100.select("span.chart-element__information")

[<span class="chart-element__information">
 <span class="chart-element__information__song text--truncate color--primary">Montero (Call Me By Your Name)</span>
 <span class="chart-element__information__artist text--truncate color--secondary">Lil Nas X</span>
 <span class="chart-element__information__delta color--secondary">
 <span class="chart-element__information__delta__text text--default">-</span>
 <span class="chart-element__information__delta__text text--last">- Last Week</span>
 <span class="chart-element__information__delta__text text--peak">1 Peak Rank</span>
 <span class="chart-element__information__delta__text text--week">1 Weeks on Chart</span>
 </span>
 </span>,
 <span class="chart-element__information">
 <span class="chart-element__information__song text--truncate color--primary">Peaches</span>
 <span class="chart-element__information__artist text--truncate color--secondary">Justin Bieber Featuring Daniel Caesar &amp; Giveon</span>
 <span class="chart-element__information__

In [19]:
# Creating a list of the artists sorted by 1st to last
top_100_artist = top_100.select("span.chart-element__information__artist")
top_100_artist = [elem.get_text() for elem in top_100_artist]

In [20]:
# Creating a list of the songs sorted by 1st to last (relative to list of artists)
top_100_song = top_100.select("span.chart-element__information__song")
top_100_song = [elem.get_text() for elem in top_100_song]

In [21]:
# Creating a list of the rankings (1-100)
top_100_rank = list(range(1,101))

In [22]:
DF = pd.DataFrame({"Rank": top_100_rank ,"Artist" : top_100_artist, "Song" : top_100_song})
DF

,Rank,Artist,Song
0,1,Lil Nas X,Montero (Call Me By Your Name)
1,2,Justin Bieber Featuring Daniel Caesar & Giveon,Peaches
2,3,Silk Sonic (Bruno Mars & Anderson .Paak),Leave The Door Open
3,4,Cardi B,Up
4,5,Olivia Rodrigo,Drivers License
...,...,...,...
95,96,Rod Wave,Shock Da World
96,97,VEDO,You Got It
97,98,Rod Wave,Sneaky Links
98,99,Dylan Scott,Nobody


In [23]:
# Lowering cases
#DF["Song"] = DF["Song"].str.lower()
#DF["Artist"] = DF["Artist"].str.lower()

In [24]:
def next_recommendation(x, y):
    '''
    This function prints the next recommendadtion then pops it from list.
    x = song list
    y = artist list
    '''
    x.pop(0)
    y.pop(0)
    output = print("Here's another one: ", x[0], "by ", y[0])
    return output

In [25]:
def get_playlist_tracks(playlist_id):
    '''
    Using sp.playlist is only limited to the first 100 tracks.
    This function returns all tracks from the spotify-playlist_id.
    '''
    results = sp.user_playlist_tracks("spotify", playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

#------------------------------------------------------------


def get_artists_from_track(track):
    '''
    
    '''
    return [artist["name"] for artist in track["artists"]]


#------------------------------------------------------------


def get_artists_from_playlist(playlist_id):
    '''
    
    '''
    tracks_from_playlist = get_playlist_tracks(playlist_id)
    return list([get_artists_from_track(track["track"])[0] for track in tracks_from_playlist])

In [81]:
def get_audio_features_from_uri(uri_list):
    audio_features = []
    counter = 0
    feat_uris = []
    for item in uri_list:
        counter += 1
        feat_uris.append(item)
        if counter == 100:
            audio_features.append(sp.audio_features(feat_uris))
            counter = 0
            feat_uris = []
            
    audio_features.append(sp.audio_features(feat_uris))
    return audio_features

In [29]:
spoti_playlists = ["5wD3jt28mHg3V6HWjOKPfY", "1fcn9HWvodDldJ5eCH4EQO", "0rRmb79AWcRCcfB69qt3ds", "7IiVIXvEEYSio3MGu2tFpP", "3EtliZeGzwZqb8nr7r5Zp6", "1IF4SWhozHVEwidHhLWwbN", "1638KZlvcvyyEJ15S8erge", "2ClhJXe7qlZSFYzqjvZnPT", "28wnlqwIjjbc0wbG1074Dx", "4jVvhTA6Xf56C1uBR48HVF"]

In [41]:
song_artists = []
for i_d in spoti_playlists:
    song_artists.append(get_artists_from_playlist(i_d))
song_artists = [subitem for item in song_artists for subitem in item]
len(song_artists)

2260

In [61]:
song_names = []
song_uris = []

for PL in spoti_playlists:
    for item in get_playlist_tracks(PL):
        #counter += 1
        song_names.append(item["track"]["name"])
        song_uris.append(item["track"]["uri"])
        #audio_features.append(sp.audio_features(item["track"]["uri"]))
        #feat_uris.append(item["track"]["uri"])
        #if counter == 50:
            #audio_features.append(sp.audio_features(feat_uris))
            #counter = 0
            #feat_uris = []

#audio_features.append(sp.audio_features(feat_uris))


In [93]:
song_features = get_audio_features_from_uri(song_uris)
song_features = [subitem for item in song_features for subitem in item]

In [108]:
features_df = pd.DataFrame(song_features)
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.681,0.372,5,-8.237,1,0.0432,0.6400,0.000000,0.1690,0.476,91.873,audio_features,1mXVgsBdtIVeCLJnSnmtdV,spotify:track:1mXVgsBdtIVeCLJnSnmtdV,https://api.spotify.com/v1/tracks/1mXVgsBdtIVe...,https://api.spotify.com/v1/audio-analysis/1mXV...,201000,4
1,0.628,0.797,11,-3.953,0,0.0596,0.0364,0.000000,0.1040,0.321,106.930,audio_features,7EI6Iki24tBHAMxtb4xQN2,spotify:track:7EI6Iki24tBHAMxtb4xQN2,https://api.spotify.com/v1/tracks/7EI6Iki24tBH...,https://api.spotify.com/v1/audio-analysis/7EI6...,215064,4
2,0.659,0.716,5,-5.673,0,0.1110,0.3300,0.000000,0.1730,0.437,148.015,audio_features,6mrKP2jyIQmM0rw6fQryjr,spotify:track:6mrKP2jyIQmM0rw6fQryjr,https://api.spotify.com/v1/tracks/6mrKP2jyIQmM...,https://api.spotify.com/v1/audio-analysis/6mrK...,212120,4
3,0.799,0.586,11,-6.735,1,0.0377,0.1620,0.000002,0.1930,0.549,120.970,audio_features,61dCUoMCg28qEBzrW6htYM,spotify:track:61dCUoMCg28qEBzrW6htYM,https://api.spotify.com/v1/tracks/61dCUoMCg28q...,https://api.spotify.com/v1/audio-analysis/61dC...,227833,4
4,0.656,0.835,9,-4.915,0,0.0278,0.1660,0.011600,0.2180,0.455,125.023,audio_features,47NTskHaXv46jp21vFj8tD,spotify:track:47NTskHaXv46jp21vFj8tD,https://api.spotify.com/v1/tracks/47NTskHaXv46...,https://api.spotify.com/v1/audio-analysis/47NT...,198439,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,0.726,0.799,1,-3.776,0,0.0678,0.1210,0.000000,0.1810,0.522,118.014,audio_features,13YIDpKrL2caqJ3YF52uzJ,spotify:track:13YIDpKrL2caqJ3YF52uzJ,https://api.spotify.com/v1/tracks/13YIDpKrL2ca...,https://api.spotify.com/v1/audio-analysis/13YI...,188636,4
2256,0.901,0.536,2,-9.075,0,0.4670,0.0312,0.000000,0.0932,0.252,132.035,audio_features,6V1lWr7s8CO5VkxXe4hBtg,spotify:track:6V1lWr7s8CO5VkxXe4hBtg,https://api.spotify.com/v1/tracks/6V1lWr7s8CO5...,https://api.spotify.com/v1/audio-analysis/6V1l...,197240,4
2257,0.778,0.695,4,-6.865,0,0.0913,0.1750,0.000000,0.1500,0.472,149.996,audio_features,3Ofmpyhv5UAQ70mENzB277,spotify:track:3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,https://api.spotify.com/v1/audio-analysis/3Ofm...,132780,4
2258,0.641,0.324,11,-5.851,1,0.0299,0.6980,0.000000,0.3280,0.273,110.184,audio_features,0E4Y1XIbs8GrAT1YqVy6dq,spotify:track:0E4Y1XIbs8GrAT1YqVy6dq,https://api.spotify.com/v1/tracks/0E4Y1XIbs8Gr...,https://api.spotify.com/v1/audio-analysis/0E4Y...,185487,4


In [113]:
song_dict = {"Song" : song_names, "Artist" : song_artists , "URI" : song_uris}
songs_df = pd.DataFrame.from_dict(song_dict)

In [120]:
#Spotify_DF = songs_df.merge(features_df, how='inner', on='uri')
#Spotify_DF.drop_duplicates()

Spotify_DF = songs_df.merge(features_df, left_index=True, right_index=True).drop(columns=["uri"])
Spotify_DF

,Song,Artist,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Too Good At Goodbyes,Sam Smith,spotify:track:1mXVgsBdtIVeCLJnSnmtdV,0.681,0.372,5,-8.237,1,0.0432,0.6400,0.000000,0.1690,0.476,91.873,audio_features,1mXVgsBdtIVeCLJnSnmtdV,https://api.spotify.com/v1/tracks/1mXVgsBdtIVe...,https://api.spotify.com/v1/audio-analysis/1mXV...,201000,4
1,Anywhere,Rita Ora,spotify:track:7EI6Iki24tBHAMxtb4xQN2,0.628,0.797,11,-3.953,0,0.0596,0.0364,0.000000,0.1040,0.321,106.930,audio_features,7EI6Iki24tBHAMxtb4xQN2,https://api.spotify.com/v1/tracks/7EI6Iki24tBH...,https://api.spotify.com/v1/audio-analysis/7EI6...,215064,4
2,Let You Down,NF,spotify:track:6mrKP2jyIQmM0rw6fQryjr,0.659,0.716,5,-5.673,0,0.1110,0.3300,0.000000,0.1730,0.437,148.015,audio_features,6mrKP2jyIQmM0rw6fQryjr,https://api.spotify.com/v1/tracks/6mrKP2jyIQmM...,https://api.spotify.com/v1/audio-analysis/6mrK...,212120,4
3,Rather Be (feat. Jess Glynne),Clean Bandit,spotify:track:61dCUoMCg28qEBzrW6htYM,0.799,0.586,11,-6.735,1,0.0377,0.1620,0.000002,0.1930,0.549,120.970,audio_features,61dCUoMCg28qEBzrW6htYM,https://api.spotify.com/v1/tracks/61dCUoMCg28q...,https://api.spotify.com/v1/audio-analysis/61dC...,227833,4
4,Book Of Love,Felix Jaehn,spotify:track:47NTskHaXv46jp21vFj8tD,0.656,0.835,9,-4.915,0,0.0278,0.1660,0.011600,0.2180,0.455,125.023,audio_features,47NTskHaXv46jp21vFj8tD,https://api.spotify.com/v1/tracks/47NTskHaXv46...,https://api.spotify.com/v1/audio-analysis/47NT...,198439,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,Too Many Nights (with JC Stewart),220 KID,spotify:track:13YIDpKrL2caqJ3YF52uzJ,0.726,0.799,1,-3.776,0,0.0678,0.1210,0.000000,0.1810,0.522,118.014,audio_features,13YIDpKrL2caqJ3YF52uzJ,https://api.spotify.com/v1/tracks/13YIDpKrL2ca...,https://api.spotify.com/v1/audio-analysis/13YI...,188636,4
2256,PEPPERMINT,Luciano,spotify:track:6V1lWr7s8CO5VkxXe4hBtg,0.901,0.536,2,-9.075,0,0.4670,0.0312,0.000000,0.0932,0.252,132.035,audio_features,6V1lWr7s8CO5VkxXe4hBtg,https://api.spotify.com/v1/tracks/6V1lWr7s8CO5...,https://api.spotify.com/v1/audio-analysis/6V1l...,197240,4
2257,Astronaut In The Ocean,Masked Wolf,spotify:track:3Ofmpyhv5UAQ70mENzB277,0.778,0.695,4,-6.865,0,0.0913,0.1750,0.000000,0.1500,0.472,149.996,audio_features,3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,https://api.spotify.com/v1/audio-analysis/3Ofm...,132780,4
2258,Afterglow,Ed Sheeran,spotify:track:0E4Y1XIbs8GrAT1YqVy6dq,0.641,0.324,11,-5.851,1,0.0299,0.6980,0.000000,0.3280,0.273,110.184,audio_features,0E4Y1XIbs8GrAT1YqVy6dq,https://api.spotify.com/v1/tracks/0E4Y1XIbs8Gr...,https://api.spotify.com/v1/audio-analysis/0E4Y...,185487,4
